# Notebook para Análisis de Precios de Toyota Corolla


## Carga del dataset


In [ ]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import mlflow

Traemos el dataframe que quedó guardado por el asset anterior


In [ ]:
from pathlib import Path

raw_path = Path("data/raw_df.csv").resolve()
raw_dataset = pd.read_csv(raw_path)

Hacemos una copia para que nos quede intacto el original por si lo necesitamos para después


In [ ]:
# Cargar dataset
df = raw_dataset.copy()

## Descripción del dataset


Vamos viendo qué onda este dataset


In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.head(15)

## EDA


Vamos viendo que **ID** es un identificador incremental, nos sirve poco y nada realmente. **Cylinders** es constante, su max y su min son 4, no varía. Hay muchas columnas bandera para indicar equipamiento, algo vamos a tener que hacer con eso.


Hacemos diferentes gráficos para visualizar las features que nos llamaron la atención


In [ ]:
from utils import barplot_feature


barplot_feature(df, "Cylinders")

Id lo revisamos con un histograma


In [ ]:
from utils import histogram_feature


histogram_feature(df, "Id")

Efectivamente, no nos sirven en absoluto


Las vamos sacando


In [ ]:
df = df.drop(columns=["Id", "Cylinders"])

Veamos qué onda las columnas que son banderas


In [ ]:
bool_cols = []
for col in df.columns:
    uniques = df[col].dropna().unique()
    if set(uniques).issubset({0, 1}) and df[col].dtype in [np.int64, np.int32, np.int8]:
        bool_cols.append(col)
print(bool_cols)

Son varias, no sabemos qué significan


Buscamos en internet qué onda esas features, encontramos esto:


1. **Id**
   Identificador único de cada vehículo en el dataset.

2. **Model**
   Versión o acabado del Toyota Corolla (por ejemplo, “Corolla D”, “Corolla GLi”).

3. **Price**
   Precio de venta del automóvil (en la unidad monetaria del dataset, p. ej., euros u otra).

4. **Age_08_04**
   Edad del coche, en años, referida al 4 de agosto de 2004 (fecha de corte utilizada para calcular antigüedad).

5. **Mfg_Month**
   Mes de fabricación del vehículo (1 = enero … 12 = diciembre).

6. **Mfg_Year**
   Año de fabricación del vehículo.

7. **KM**
   Kilometraje recorrido por el coche (en kilómetros).

8. **Fuel_Type**
   Tipo de combustible:

   - “Petrol” (gasolina)
   - “Diesel” (diésel)
   - “CNG” (gas natural comprimido)

9. **HP**
   Potencia del motor en caballos de fuerza (Horse Power).

10. **Met_Color**
    Indicador de pintura metálica (1 = sí, 0 = no).

11. **Automatic**
    Tipo de transmisión (1 = automática, 0 = manual).

12. **cc**
    Cilindrada del motor en centímetros cúbicos (cm³).

13. **Doors**
    Número de puertas del vehículo.

14. **Cylinders**
    Número de cilindros del motor.

15. **Gears**
    Número de marchas de la transmisión.

16. **Quarterly_Tax**
    Importe del impuesto de circulación o matriculación que se paga trimestralmente (en la misma unidad monetaria que Price).

17. **Weight**
    Peso del vehículo en kilogramos.

18. **Mfr_Guarantee**
    Garantía del fabricante (1 = incluida, 0 = no incluida).

19. **BOVAG_Guarantee**
    Garantía ofrecida por BOVAG (asociación neerlandesa de concesionarios) (1 = incluida, 0 = no).

20. **Guarantee_Period**
    Duración de la garantía (en años).

21. **ABS**
    Sistema antibloqueo de frenos (Anti-lock Braking System) (1 = sí, 0 = no).

22. **Airbag_1**
    Airbag para el conductor (1 = sí, 0 = no).

23. **Airbag_2**
    Airbag para el pasajero delantero (1 = sí, 0 = no).

24. **Airco**
    Aire acondicionado (1 = sí, 0 = no).

25. **Automatic_airco**
    Control automático de la temperatura (climatizador) (1 = sí, 0 = no).

26. **Boardcomputer**
    Ordenador de a bordo (display con información de viaje, consumo, etc.) (1 = sí, 0 = no).

27. **CD_Player**
    Reproductor de CD (1 = sí, 0 = no).

28. **Central_Lock**
    Cierre centralizado de puertas (1 = sí, 0 = no).

29. **Powered_Windows**
    Elevalunas eléctricos (1 = sí, 0 = no).

30. **Power_Steering**
    Dirección asistida (1 = sí, 0 = no).

31. **Radio**
    Radio estándar (1 = sí, 0 = no).

32. **Mistlamps**
    Faros antiniebla (1 = sí, 0 = no).

33. **Sport_Model**
    Edición deportiva del modelo (1 = sí, 0 = no).

34. **Backseat_Divider**
    Separador o consola entre asientos traseros (1 = sí, 0 = no).

35. **Metallic_Rim**
    Llantas de aleación metálica (1 = sí, 0 = no).

36. **Radio_cassette**
    Radio con reproductor de casete (1 = sí, 0 = no).

37. **Tow_Bar**
    Enganche de remolque o bola de remolque (1 = sí, 0 = no).


Veamos qué peso tienen sobre la variable objetivo, podemos hacer una matriz de correlación solamente de las banderas, a ver si son importantes


In [ ]:
from utils import show_correlation_matrix


matrix = show_correlation_matrix(df[bool_cols + ["Price"]])

Lo primero que llama la atención es la correlación tan fuerte que existe entre radio y radio_cassette. Tiene sentido. Nos podemos quedar con una nomás, ambas tienen el mismo peso sobre Price.


In [ ]:
# Contar filas donde Radio y Radio_cassette difieren
diferentes = (df["Radio"] != df["Radio_cassette"]).sum()
print(f"Número de filas con valores distintos entre 'Radio' y 'Radio_cassette': {diferentes}")
print(f"Número de filas con valores iguales entre 'Radio' y 'Radio_cassette': {len(df) - diferentes}")

Prácticamente siempre que hay radio hay radio cassette, borramos una


In [ ]:
df = df.drop(columns=["Radio_cassette"])

Siguen habiendo otras correlaciones más débiles, capaz vale la pena tratarlas.


Seguimos mirando las variables


In [ ]:
df.describe().T

El mes de manufactura no debería servir así como está planteado, lo verificamos:


In [ ]:
matrix = show_correlation_matrix(df[["Mfg_Month", "Mfg_Year", "Age_08_04", "Price"]])

Como sospechábamos, el mes por sí solo no tiene ningún peso sobre el precio. Y columnas como año y edad son mucho más importantes. Nos deshacemos de la columna mes, no hay nada que rescatarle.


In [ ]:
df = df.drop(columns=["Mfg_Month"])

Vamos explorando otro lado, veamos lo de automatic


Empezamos con un histograma, para ver cómo está distribuida esta clase


In [ ]:
barplot_feature(df, "Automatic")

In [ ]:
# Calcular porcentaje de filas con Automatic igual a 1 (True) y 0 (False)
total = len(df)
pct_true = (df["Automatic"] == 1).sum() / total * 100
pct_false = (df["Automatic"] == 0).sum() / total * 100

print(f"Porcentaje con Automatic=True: {pct_true:.2f}%")
print(f"Porcentaje con Automatic=False: {pct_false:.2f}%")

In [ ]:
from utils import scatter_feature, violinplot_feature


violinplot_feature(df, "Automatic")

Veamos de qué año son los automáticos, capaz por edad valen menos


In [ ]:
precio_promedio_automaticos = df[df["Automatic"] == 1].groupby("Mfg_Year")["Price"].mean().sort_index()
precio_promedio_no_automaticos = df[df["Automatic"] == 0].groupby("Mfg_Year")["Price"].mean().sort_index()
precios_promedio = pd.DataFrame({
    "Automático": precio_promedio_automaticos,
    "Manual": precio_promedio_no_automaticos
})

print(precios_promedio)

Bueno, algo de influencia se ve, alrededor de mil dólares más.


Pero la clase está muy desbalanceada, veremos más adelante si vale la pena rescatar esta feature.


Y por el violin plot podemos ver que en todos los quintiles de precio siguen siendo mayoría los autos manuales (Automatic=0), no le encontramos valor sifnificativo a la variable Automatic. Echamos un último vistazo a la matriz de correlación que hicimos arriba.


In [ ]:
df = df.drop(columns=["Automatic"])

Aquí queremos ver si existen features que sean iguales entre sí, es decir si tienen todas el mismo valor 1 al mismo tiempo, y ver si se puede crear otra feature a partir de ellas


In [ ]:
from collections import defaultdict

# Calculamos bools cols devuelta porque ya borramos cosas antes
bool_cols = []
for col in df.columns:
    uniques = df[col].dropna().unique()
    if set(uniques).issubset({0, 1}) and df[col].dtype in [np.int64, np.int32, np.int8]:
        bool_cols.append(col)

groups = defaultdict(list)
for col in bool_cols:
    key = tuple(df[col].fillna(-1))
    groups[key].append(col)

duplicate_groups = [cols for cols in groups.values() if len(cols) > 1]

if duplicate_groups:
    print("Las siguientes columnas binarias son idénticas en todas las filas:")
    for group in duplicate_groups:
        print("  -", ", ".join(group))
else:
    print("No se encontraron columnas binarias idénticas entre sí.")

Bueno a primera vista no hay observaciones con todas las banderas con el mismo valor, más adelante vamos a seguir viendo esto.


Le peguemos un vistazo a met_color a ver en qué consiste


In [ ]:
from utils import barplot_features_batch, histogram_by_batch


cols = ["Met_Color", "Sport_Model", "Mistlamps"]

barplot_features_batch(df, cols)

In [ ]:
from utils import violinplot_features_batch


violinplot_features_batch(df, cols)

La variable Year tenía apenas más correlación con el precio que Age en meses, pensamos en armar un age expresado en años para tener esa mayor correlación y reducir la escala de ese dato


In [ ]:
df["Age_in_Years_08_04"] = (df["Age_08_04"] / 12).astype(int)
barplot_feature(df, "Age_in_Years_08_04")
histogram_feature(df, "Age_in_Years_08_04")
violinplot_feature(df, "Age_in_Years_08_04")

In [ ]:
matrix = show_correlation_matrix(df[["Age_in_Years_08_04", "Age_08_04", "Price"]])

Sigue teniendo más correlación Age en meses


De momento descartamos esto de expresar age de otra forma


Las columnas binarias son muchas, y no agregan mucho valor por sí solas


In [ ]:
bool_cols

Veamos la correlación que tiene cada una


In [ ]:
matrix = show_correlation_matrix(df[bool_cols + ["Price"]])

De todas esas boolenas, algunas podemos considerarlas como equipamiento


In [ ]:
equipment_cols = [
    "ABS",
    "Airbag_1",
    "Airbag_2",
    "Airco",
    "Automatic_airco",
    "Boardcomputer",
    "CD_Player",
    "Central_Lock",
    "Powered_Windows",
    "Power_Steering",
    "Radio",
    "Mistlamps",
    "Backseat_Divider",
    "Tow_Bar",
]

Vemos qué tal están distribuidas estas variables


In [ ]:
barplot_features_batch(df, equipment_cols)

Esto es el primer experimento, probamos hacer la suma ponderada de las ocurrencias de los equipamientos con su correlación con la variable objetivo.


In [ ]:
correlations = df[equipment_cols + ["Price"]].corr()["Price"].abs().drop("Price")
weights = correlations / correlations.sum()
df["Equipment_Score"] = df[equipment_cols].mul(weights).sum(axis=1)

In [ ]:
matrix = show_correlation_matrix(df[["Equipment_Score", "Price"]])

In [ ]:
from utils import boxplot_feature


boxplot_feature(df, "Equipment_Score")

In [ ]:
scatter_feature(df, "Equipment_Score")

In [ ]:
histogram_feature(df, "Equipment_Score", bins=100)

In [ ]:
violinplot_feature(df, "Equipment_Score")

Tratamos de crear otra feature, una que directamente cuente los equipamientos


In [ ]:
df["Equipment_Count"] = df[equipment_cols].sum(axis=1)
df["Equipment_Count"].plot(kind="box", figsize=(10, 5))

In [ ]:
scatter_feature(df, "Equipment_Count")

In [ ]:
histogram_feature(df, "Equipment_Count")

Probemos otra cosa, una feature que se fija si el auto tiene equipamiento premium. Del conocimiento del dominio, equipamiento premium sería climatizador, computadora a bordo


In [ ]:
df["Premium_Equipment"] = df["Automatic_airco"] | df["Boardcomputer"]

In [ ]:
matrix = show_correlation_matrix(
    df[["Premium_Equipment", "Equipment_Count", "Equipment_Score", "Price"]]
)

In [ ]:
violinplot_feature(df, "Premium_Equipment")

Veamos cómo se distribuye premium equipment


In [ ]:
barplot_feature(df, "Premium_Equipment")

Está sesgado a la izquierda **Equipment_Score**, lo cuál es una pena porque tenemos 0,68 de correlación.


Y su scatter plot


In [ ]:
scatter_feature(df, "Equipment_Score")

Creemos que al principio, cuantos más equipamientos mejor, pero cuando ya se tiene una cierta base, se estabiliza el precio un poco, por el scatter plot creemos que una exponencial se ajustaría mejor


Veamos si con una transformación de raiz cuadrada podemos forzar su distribución a parecerse más a una normal


In [ ]:
df["Equip_Score_sqrt"] = np.sqrt(df["Equipment_Score"])

In [ ]:
from utils import boxplot_feature


histogram_feature(df, "Equip_Score_sqrt")
boxplot_feature(df, "Equip_Score_sqrt")
violinplot_feature(df, "Equip_Score_sqrt")

In [ ]:
matrix = show_correlation_matrix(df[["Equip_Score_sqrt", "Price"]])

In [ ]:
def tag_equipment(score):
    if score <= 0.2:
        return "Bajo"
    elif score <= 0.6:
        return "Medio"
    else:
        return "Alto"


df["Equipment_Level"] = df["Equipment_Score"].apply(tag_equipment)

dummies = pd.get_dummies(df["Equipment_Level"], prefix="Equip_Level").astype(int)
df = pd.concat([df, dummies], axis=1)

In [ ]:
matrix = show_correlation_matrix(
    df[["Equip_Level_Bajo", "Equip_Level_Medio", "Equip_Level_Alto", "Price"]]
)

Puede que Equip_Level_Alto tenga alguna utilidad


Veamos cómo está distribuida


In [ ]:
equip_levels = ["Equip_Level_Bajo", "Equip_Level_Medio", "Equip_Level_Alto"]

barplot_features_batch(df, equip_levels)

Nada útil. Vamos descartando nuestros intentos


In [ ]:
df = df.drop(
    columns=equip_levels + ["Equipment_Level", "Equipment_Score", "Premium_Equipment"]
)

Con todo, vemos qué tenemos hasta el momento


In [ ]:
matrix = show_correlation_matrix(df)

Hay una feature para indicar si tiene un tipo de garantía, otra feature para indicar que tiene del otro tipo de garantía, y otra feature más que indica qué período de garantía.


In [ ]:
mask = (
    (df["BOVAG_Guarantee"] == 0)
    & (df["Mfr_Guarantee"] == 0)
    & (df["Guarantee_Period"] != 0)
)
df.loc[mask, ["BOVAG_Guarantee", "Mfr_Guarantee", "Guarantee_Period"]]

In [ ]:
df["Has_Guarantee"] = (df["BOVAG_Guarantee"] == 1) | (df["Mfr_Guarantee"] == 1)

In [ ]:
df["Has_Guarantee"].value_counts()

In [ ]:
df["True_Guarantee_Period"] = (
    df["Guarantee_Period"].where(df["Has_Guarantee"], other=0).astype(int)
)

In [ ]:
df["True_Guarantee_Period"].value_counts()

Esta exploración de Garantías no nos llevó a ningún lado.


**Airco** está muy balanceada, podría sernos útil. Veamos el gráfico de violín para entender mejor el impacto de estos equipamientos en los diferentes quintiles de precio.


In [ ]:
violinplot_feature(df, "Airco")

Lo vemos muy positivo a Airco. Está bien distribuido, se le ve impacto en los quintiles, tiene 0,43 de correlación (de momento). Muy positivo.


Investiguemos Boardcomputer, porque tiene una correlación alta de 0,60 (de momento).


In [ ]:
violinplot_feature(df, "Boardcomputer")

**Boardcomputer** vemos que explica bien el Price, en quintiles bajos es muy frecuente su ausencia, y en quintiles altos domina su presencia. Está desbalanceada lametablemente. Veremos si la podemos rescatar un poco.


Observamos que tienen una alta correlación las variables Central_Lock y Powered_Windows. Por lo que entendemos del problema, creemos que ambas características tienen sentido en conjunto, por ejemplo en aquellos autos en los que al bloquearlos con el cierre centralizado automáticamente levantan los vidrios si es que estaban bajados.


Veamos si esto sucede y con qué frecuencia:


In [ ]:
# Contar filas donde Powered_Windows = 1 y Central_Lock = 0
pw_sin_cl = ((df["Powered_Windows"] == 1) & (df["Central_Lock"] == 0)).sum()

# Contar filas donde Central_Lock = 1 y Powered_Windows = 0
cl_sin_pw = ((df["Central_Lock"] == 1) & (df["Powered_Windows"] == 0)).sum()

# Contar filas donde ambas son 1
ambos = ((df["Powered_Windows"] == 1) & (df["Central_Lock"] == 1)).sum()

print(f"Filas con Powered_Windows=1 y Central_Lock=0: {pw_sin_cl}")
print(f"Filas con Central_Lock=1 y Powered_Windows=0: {cl_sin_pw}")
print(f"Filas con Powered_Windows=1 y Central_Lock=1: {ambos}")

Ya que estamos, veamos su influencia sobre los quintiles de Precio


In [ ]:
violinplot_feature(df, "Powered_Windows")
violinplot_feature(df, "Central_Lock")

En base a esto, considerando su fuerte correlación, considerando su peso sobre el Price, y considerando la correlación que tienen con la variable objetivo, elegimos quedarnos con Powered_Windows


Central Lock se va:


In [ ]:
df = df.drop(columns=["Central_Lock"])

Siendo que nos quedamos con Airco, Powered_Windows y Boardcomputer, aquellas que puedan presentar colinealidad con estas features elegidas serán borradas.


In [ ]:
matrix = show_correlation_matrix(df)

Board_Computer y CD_Player tienen alta correlación, sacamos CD_Player


In [ ]:
df = df.drop(columns=["CD_Player"])

Powered_Windows con Mistlamps tienen alta correlación, se van Mistlamps


In [ ]:
df = df.drop(columns=["Mistlamps"])

Echemos un vistazo a CC


In [ ]:
barplot_feature(df, "cc")

16000 de cilindrada es un motor de 16 litros, seguramente está mal imputado ese dato, y corresponde a la clase o valor más frecuente, que serían 1600cc, o un motor 1.6.


In [ ]:
# Find and correct the row where cc is 16000
df.loc[df["cc"] == 16000, "cc"] = 1600

1975 y 1995 son muy próximos a 2000, y valores tan específicos de cilindrada o pueden ser casos muy particulares o errores de imputación. En todo caso no nos aporta nada estas clases tan específicas y con tan poca frecuencia, por lo que las forzamos.


In [ ]:
df.loc[df["cc"].isin([1975, 1995]), "cc"] = 2000

1598 y 1587 los correjimos para que pertenezcan a 1600


In [ ]:
df.loc[df["cc"].isin([1598, 1587]), "cc"] = 1600

1398 a 1400


In [ ]:
df.loc[df["cc"] == 1398, "cc"] = 1400

1332 se redondea para abajo, pero nos terminaría quedando una clase con una sola ocurrencia, de momento lo hacemos, ya veremos cómo mejorar esta feature


In [ ]:
df.loc[df["cc"] == 1332, "cc"] = 1300

Y ahora veamos el violin plot de cc a ver si notamos algún patrón


In [ ]:
violinplot_feature(df, "cc")

Observamos, para cada quintil, 3 concentraciones de valores con medias en 1600, otras oscilando entre 1200 y 1400, y otras alrededor de 2000. Podemos interpretar que hay 3 tipos de motores, 1600 es un motor de tamaño medio, menos de 1600 es un motor chico, y más de 1600 es un motor grande. Veamos si agrupandolo así podemos lograr un feature que aporte más al modelo.


In [ ]:
def classify_engine_size(cc):
    if cc < 1500:
        return 1  # Motor Chico
    elif cc <= 1800:
        return 2  # Motor Medio
    else:
        return 3  # Motor Grande


# Create new feature as an ordinal variable
df["Engine_Size"] = df["cc"].apply(classify_engine_size)
df["Engine_Size"]

In [ ]:
df.dtypes

In [ ]:
violinplot_feature(df, "Engine_Size")
matrix = show_correlation_matrix(df[["Engine_Size", "Price"]])

No parece muy útil, la quitamos


In [ ]:
df = df.drop(columns=["Engine_Size"])

Exploremos quarterly tax


In [ ]:
df["Quarterly_Tax"].value_counts()

In [ ]:
barplot_feature(df, "Quarterly_Tax")
histogram_feature(df, "Quarterly_Tax", bins=13)
violinplot_feature(df, "Quarterly_Tax")

In [ ]:
# Count observations that would be removed
mask = (df["Quarterly_Tax"] < 50) | (df["Quarterly_Tax"] > 150)
removed_count = mask.sum()

# Get total observations and calculate percentage
total = len(df)
removed_pct = (removed_count / total) * 100

print(f"Total observations: {total}")
print(f"Would remove: {removed_count} observations ({removed_pct:.2f}%)")

In [ ]:
# Create a filtered view without modifying original df
filtered_df = df[~mask]

# Create bar plot
histogram_feature(filtered_df, "Quarterly_Tax")
violinplot_feature(filtered_df, "Quarterly_Tax")

# Show correlation matrix for filtered data
matrix = show_correlation_matrix(filtered_df[["Quarterly_Tax", "Price"]])

Podemos dividir en dos, impuestos altos e impuestos bajos


In [ ]:
df["Quarterly_Tax"].mean()

In [ ]:
df["High_Tax"] = (df["Quarterly_Tax"] > 80).astype(int)

In [ ]:
violinplot_feature(df, "High_Tax")
matrix = show_correlation_matrix(df[["High_Tax", "Price"]])

Veamos qué features son las más correlacionadas con el Price hasta el momento.


In [ ]:
top_corr = matrix["Price"].abs().sort_values(ascending=False).iloc[1:16]
print(top_corr)

Exploremos Horse Power


In [ ]:
barplot_feature(df, "HP")
histogram_feature(df, "HP", bins=50)
violinplot_feature(df, "HP")

Probamos borrando esos outliers de HP


In [ ]:
# Create a copy of the dataframe
df_filtered = df.copy()

# Remove rows where HP > 150
mask = df_filtered["HP"] <= 150
df_filtered = df_filtered[mask]

# Show some statistics before and after
print(f"Original df shape: {df.shape}")
print(f"Filtered df shape: {df_filtered.shape}")
print("\nHP statistics in original df:")
print(df["HP"].describe())
print("\nHP statistics in filtered df:")
print(df_filtered["HP"].describe())

matrix = show_correlation_matrix(df_filtered[["HP", "Price"]])
histogram_feature(df_filtered, "HP")

No vemos mucho que rescatarle a HP, poca correlación


También podemos buscar una relación entre la potencia y el peso del vehículo


In [ ]:
df["Weight_HP_Ratio"] = df["Weight"] / df["HP"]
histogram_feature(df, "Weight_HP_Ratio")

Otra podría ser el nivel de uso, cuántos kilometros tiene para la edad que tiene


In [ ]:
df["KM_per_Year"] = df["KM"] / df["Age_in_Years_08_04"].replace(0, 1)
df["KM_per_Month"] = df["KM"] / df["Age_08_04"].replace(0, 1)

# Mostrar correlación con Price
correlation_year = df["KM_per_Year"].corr(df["Price"])
correlation_month = df["KM_per_Month"].corr(df["Price"])
print(f"Correlación de KM_per_Year con Price: {correlation_year:.4f}")
print(f"Correlación de KM_per_Month con Price: {correlation_month:.4f}")
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(df["KM_per_Year"], bins=50, kde=True, ax=axes[0], color="green")
axes[0].set_title(f"Distribución de KM por Año de Uso (r={correlation_year:.4f})")
axes[0].set_xlabel("KM/Año")

sns.histplot(df["KM_per_Month"], bins=50, kde=True, ax=axes[1], color="blue")
axes[1].set_title(f"Distribución de KM por Mes de Uso (r={correlation_month:.4f})")
axes[1].set_xlabel("KM/Mes")

plt.tight_layout()
plt.show()

usage_correlations = show_correlation_matrix(
    df[["KM_per_Year", "KM_per_Month", "KM", "Age_08_04", "Price"]]
)

Casi nada de correlación, no es por aquí.


In [ ]:
df = df.drop(columns=["KM_per_Year", "KM_per_Month"])

In [ ]:
len(df.columns)

Por el sentido común, y por el scatter plot de abajo, creemos que el kilometraje y el precio no tienen una relación lineal, el precio en kilometrajes bajos es muy sensible, pero en kilometrajes altos se estabiliza


Se nos ocurren dos alternativas. Una es considerarla cuadrática e intentar mejorarla con un sqrt. La otra es tratar de aplicar una hipérbola de una función racional. Veamos qué tal quedan con eso.


In [ ]:
scatter_feature(df, "KM", "Price")
violinplot_feature(df, "KM")
histogram_feature(df, "KM", bins=100)

Aplicamos la raíz cuadrada


In [ ]:
df["KM_Sqrt"] = np.sqrt(df["KM"])
scatter_feature(df, "KM_Sqrt")
violinplot_feature(df, "KM_Sqrt")
histogram_feature(df, "KM_Sqrt", bins=100)

Veamos con un logaritmo


In [ ]:
df["KM_Log"] = np.log1p(df["KM"])
scatter_feature(df, "KM_Log")
violinplot_feature(df, "KM_Log")
histogram_feature(df, "KM_Log", bins=100)

Vemos la correlación


In [ ]:
matrix = show_correlation_matrix(df[["KM_Sqrt", "KM_Log", "Price"]])

KM_Log fue un error, nos deshacemos


In [ ]:
df = df.drop(columns=["KM_Log"])

Exploremos un poco weight


In [ ]:
histogram_feature(df, "Weight")
scatter_feature(df, "Weight", "Price")
violinplot_feature(df, "Weight")

Más allá de que haya outliers, no se ve una relación significativa con el price y con los quintiles, no le vemos mucho valor a weight.


In [ ]:
# Calcular la correlación de todas las columnas numéricas con Weight
weight_corr = df.corr(numeric_only=True)["Weight"].abs().sort_values(ascending=False)
print(weight_corr.head(15))

Veamos cómo está distribuido Age


In [ ]:
histogram_feature(df, "Age_08_04", bins=50)
scatter_feature(df, "Age_08_04")

También nos quedó pendiente la feature fuel type, vamos a hacerle dummies porque es categórica


In [ ]:
fuel_dummies = pd.get_dummies(
    df["Fuel_Type"], prefix="Fuel_Type", dtype=int, drop_first=True
)
df = pd.concat([df, fuel_dummies], axis=1)

In [ ]:
violinplot_feature(df, "Fuel_Type_Petrol")

In [ ]:
matrix = show_correlation_matrix(df[["Fuel_Type_Diesel", "Fuel_Type_Petrol", "Price"]])

Qué más nos falta ver


In [ ]:
df.columns
df.dtypes.head()

In [ ]:
matrix = show_correlation_matrix(df)

Veamos una combinación de Airbags


In [ ]:
# Create new feature combining airbags
df["Total_Airbags"] = df["Airbag_1"] + df["Airbag_2"]

# Create violin plot for Total_Airbags
violinplot_feature(df, "Total_Airbags")

# Create bar plot for Total_Airbags
barplot_feature(df, "Total_Airbags")

# Show correlation with Price
matrix = show_correlation_matrix(df[["Total_Airbags", "Price"]])

In [ ]:
df.dtypes

## Limpieza


Ahora mismo nuestro dataset se ve así


In [ ]:
df.describe().T

In [ ]:
df.dtypes.iloc[15:]

Del trabajo anterior en el que tratamos de crear nuevas features y mejorarlas, vamos a ver cuáles son las mejores candidatas a limpiar


Tomamos las variables más correlacionadas con Price


In [ ]:
correlations = df.corr(numeric_only=True)["Price"].abs().sort_values(ascending=False)

top_corr_vars = correlations.drop("Price").head(15)
print(top_corr_vars)

Muchas de las variables más correlacionadas de este top son variaciones de la misma información, nos quedamos con una de cada tipo, eligiendo por su distribución en general, y por su correlación


Age_Sqrt, Mfg_Year, Age_08_04, Age_in_Years_08_04 son todas variaciones de la edad del vehículo. Nos quedamos con Age_Sqrt que es la mejor por el análisis que le hicimos, de todas formas aquí van sus gráficos de nuevo


In [ ]:
# from utils import plot_feature_analysis

# plot_feature_analysis(df, "Age_Sqrt")

Para el equipment tenemos el equipment alto, el equipment premium, el equipment count. De todos, el que más utilidad nos puede dar es Equipment_Score, por su correlación, y su distribución más similar a una normal.


In [ ]:
# plot_feature_analysis(df, "Equipment_Score")

Para el kilometraje nos quedamos con su variante de raíz cuadrada, que explica mejor el comportamiento del precio y que tiene una distribución muy próxima a una normal


In [ ]:
# plot_feature_analysis(df, "KM_Sqrt")

Para el peso nos quedamos con su versión transformada con logaritmo, porque aunque tengan correlación muy similar, la transformada tiene una mejor distribución


In [ ]:
# plot_feature_analysis(df, "Weight_Log", bins=100)

Vamos sacando todo lo que ya no nos sirve


In [ ]:
# matrix = show_correlation_matrix(df)

Vamos a tratar de limpiar lo más que podamos estas variables en orden de importancia


### Price


In [ ]:
# plot_feature_analysis(df, "Price")

Price presenta varios outliers, algunos pocos vehiculos con precios muy elevados. Más allá de eso, la distribución de price parece rescatable, vamos a empezar por limpiar outliers usando IQR


In [ ]:
# from utils import clean_outliers_iqr


# df = df[clean_outliers_iqr(df["Price"])[1]]
# plot_feature_analysis(df, "Price")

hemos probado con zscore y la distribución resultante no es adecuada, vamos a probar borrando directamente los precios por encima de cierto valor


In [ ]:
# df = df[df["Price"] <= 15000]
# plot_feature_analysis(df, "Price", bins=50)

### Age_Sqrt


In [ ]:
# plot_feature_analysis(df, "Age_Sqrt")

Age_Sqrt presenta varios outliers, empecemos limpiandolo para ver cómo queda la distribución


In [ ]:
# df = df[clean_outliers_iqr(df["Age_Sqrt"])[1]]
# plot_feature_analysis(df, "Age_Sqrt")

Aplicamos zscore


In [ ]:
# from utils import clean_outliers_zscore


# df = df[clean_outliers_zscore(df["Age_Sqrt"])[1]]
# plot_feature_analysis(df, "Age_Sqrt")

## Equipment_Score


In [ ]:
# plot_feature_analysis(df, "Equipment_Score")

Equipment_Score no presenta outliers, lo dejamos como está porque un zscore no dió resultados satisfactorios


### KM_Sqrt


In [ ]:
# plot_feature_analysis(df, "KM_Sqrt")

KM_sqrt está muy bien en primera instancia, vamos a aplicar un zscore ya que la distribución es casi normal


In [ ]:
# df = df[clean_outliers_zscore(df["KM_Sqrt"])[1]]
# plot_feature_analysis(df, "KM_Sqrt")

No más tratamiento


### Weight_Log


In [ ]:
# plot_feature_analysis(df, "Weight_Log")

Weight presenta un sesgo a la derecha, aplicaremos un IQR para ir limpiando outliers


In [ ]:
# df = df[clean_outliers_iqr(df["Weight_Log"])[1]]
# plot_feature_analysis(df, "Weight_Log")

### HP


In [ ]:
# plot_feature_analysis(df, "HP", bins=50)

### HP


No parece rescatable, y no derivamos ninguna feature de aquí


### Quarterly_Tax


In [ ]:
# plot_feature_analysis(df, "Quarterly_Tax", bins=50)

No se puede recuperar esta variable


### Resto de variables


No tiene sentido seguir limpiando, estamos en la marca de 1000 observaciones y las variables que quedan tienen una correlación muy baja


In [ ]:
# matrix = show_correlation_matrix(df)

In [ ]:
df.dtypes

# Guardar dataset limpio


In [ ]:
# Drop non-numeric columns from the dataframe
numeric_df = df.select_dtypes(include=["int64", "float64", "bool"])

# Print final shape
print(f"Original shape: {df.shape}")
print(f"Numeric shape: {numeric_df.shape}")

df = numeric_df.copy()

Original shape: (1436, 41)
Numeric shape: (1436, 38)


In [ ]:
from pathlib import Path


clean_path = Path("data/clean_df.csv").resolve()
df.to_csv(clean_path, index=False)
clean_path

In [ ]:
# orig_doors = pd.to_numeric(df["Doors"], errors="coerce").astype("Int64")

# raw = df["Model_Clean"].str.extract(
#     r"(\d(?:/\d)?)(?=\s*[-\s]?(?:Doors?|Drs?))", expand=False
# )


# def normalize(val):
#     if pd.isna(val):
#         return pd.NA
#     return max(map(int, val.split("/"))) if "/" in val else int(val)


# df["Doors_extracted"] = raw.apply(normalize).astype("Int64")
# df["Doors_extracted"].value_counts(dropna=False)

In [ ]:
# df["Doors"].value_counts(dropna=False)

In [ ]:
# orig_doors = pd.to_numeric(df["Doors"], errors="coerce").astype("Int64")

# raw = df["Model_Clean"].str.extract(
#     r"(\d(?:/\d)?)(?=\s*[-\s]?(?:Doors?|Drs?))", expand=False
# )


# def normalize(val):
#     if pd.isna(val):
#         return pd.NA
#     return max(map(int, val.split("/"))) if "/" in val else int(val)


# df["Doors_extracted"] = raw.apply(normalize).astype("Int64")

# body_map = {
#     "Sedan": 4,
#     "Coupe": 3,
#     "Convertible": 3,
#     "Hatchback": 5,
#     "Hatchb": 5,
#     "Wagon": 5,
#     "Stationwagen": 5,
#     "Sw": 5,
#     "Station": 5,
#     "Mpv": 5,
#     "Verso": 5,
# }
# mask_inf = df["Doors_extracted"].isna() & df["Body_Style"].notna()
# df.loc[mask_inf, "Doors_extracted"] = df.loc[mask_inf, "Body_Style"].map(body_map)
# manual_map = {"Gli": 4, "XLi": 4, "E-Four": 4, "XEi": 4, "16V": 4}


# def assign_manual(text):
#     for key, doors in manual_map.items():
#         if key.lower() in text.lower():
#             return doors
#     return pd.NA


# mask_man = df["Doors_extracted"].isna()
# df.loc[mask_man, "Doors_extracted"] = (
#     df.loc[mask_man, "Model_Clean"].apply(assign_manual).astype("Int64")
# )
# combined = pd.concat(
#     [orig_doors.fillna(0), df["Doors_extracted"].fillna(0)], axis=1
# ).max(axis=1)
# df["Doors"] = combined.replace({0: pd.NA}).astype("Int64")

# five_styles = {
#     "Hatchback",
#     "Hatchb",
#     "Wagon",
#     "Stationwagen",
#     "Sw",
#     "Station",
#     "Mpv",
#     "Verso",
# }


# def force_3_or_5(row):
#     d = row["Doors"]
#     if d in (3, 5):
#         return d
#     return 5 if row["Body_Style"] in five_styles else 3


# df["Doors"] = df.apply(force_3_or_5, axis=1).astype("Int64")
# df.drop(columns=["Doors_extracted"], inplace=True)

# print(df[["Model_Clean", "Doors"]])
# print("\nDistribución de Doors:")
# print(df["Doors"].value_counts(dropna=False))

Después de corregir la feature Doors veremos cómo quedó


In [ ]:
# df["Doors"].value_counts()

In [ ]:
# hist = histogram(df["Doors"], title="Doors")

Quedó bastante bien


En la siguiente celda, retiramos las features auxiliares derivadas de model para que no interfieran en el dataset


In [ ]:
# df.drop(
#     columns=["Model", "Model_Clean", "Model_Len", "Model_Words"],
#     inplace=True,
#     axis=1,
#     errors="ignore",
# )
# print(df.columns.tolist())

Ahora veremos qué tal las features nuevas en una matriz de correlación


In [ ]:
# df["Body_Style"].value_counts(dropna=False)

In [ ]:
# df[
#     [
#         "Doors",
#         "Brand",
#         "Series",
#         "Engine_Size",
#         "Engine_Tech",
#         "Body_Style",
#         "Trim",
#         "Price",
#     ]
# ]

In [ ]:
# df["Engine_Tech"].value_counts(dropna=False)

In [ ]:
# df["Body_Style"].value_counts()

Como notamos


In [ ]:
# matrix = show_correlation_matrix(
#     df[
#         [
#             "Doors",
#             "Brand",
#             "Series",
#             "Engine_Size",
#             "Engine_Tech",
#             "Body_Style",
#             "Trim",
#             "Price",
#         ]
#     ]
# )

## Eliminación de valores nulos


In [ ]:
# # Seleccionar columnas numéricas
# numeric_df = df.select_dtypes(include="number")

# # Verificar si hay valores negativos
# negativos_bool = (numeric_df < 0).any()

# # Convertir a DataFrame con nombre de columna y booleano
# negativos_df = negativos_bool.reset_index()
# negativos_df.columns = ["columna", "tiene_valores_negativos"]

# # Mostrar el resultado
# negativos_df

# Limpieza de datos


In [ ]:
# # Sacamos las columnas que no nos interesan
# df = df.drop(columns=["Id"], axis=1, errors="ignore")
# df

In [ ]:
# # 1. Obtener el número de nulos por columna
# null_counts = df.isnull().sum()

# # 2. Filtrar solo columnas con al menos un nulo
# null_counts = null_counts[null_counts > 0]

# # 3. Mostrar el resultado
# print(null_counts)

## Detección de valores duplicados


In [ ]:
# def duplicados_con_indices(df):
#     resultado = []

#     for i in range(len(df)):
#         fila_actual = df.iloc[i]
#         duplicado_en = False

#         for j in range(i):
#             if df.iloc[j].equals(fila_actual):
#                 duplicado_en = j
#                 break

#         resultado.append({"Fila": i, "Duplicado_de": duplicado_en})

#     # Convertir la lista de resultados en un DataFrame
#     df_resultado = pd.DataFrame(resultado)
#     return df_resultado


# df_re = duplicados_con_indices(df)

In [ ]:
# # Crear una columna 'dup_col' que marca los duplicados según df_re
# dup_col = "Duplicado_de"
# df[dup_col] = False
# for idx, orig in zip(df_re["Fila"], df_re["Duplicado_de"]):
#     df.at[idx, dup_col] = orig

# # 1) Filtrar las filas marcadas
# df_dup = df.loc[df[dup_col] != False]

# # 2) Sacar índices de duplicados y de sus originales
# dup_idxs = df_dup.index.tolist()
# orig_idxs = df_dup[dup_col].tolist()

# # 3) Unión única y ordenada de índices
# all_idxs = sorted(set(dup_idxs + orig_idxs))

# # 4) Extraer esas filas completas
# pd.set_option("display.max_columns", None)
# df_pairs = df.loc[all_idxs]

# # 5) Mostrar resultado
# df_pairs

In [ ]:
# # Verificar si hay columnas constantes
# columnas_cte = df.columns[df.nunique() == 1]
# print("Columnas constantes:", columnas_cte.tolist())

In [ ]:
# # borrar columnas constantes
# df.drop(columns=["Cylinders", "Brand", "Series"], axis=1, inplace=True)
# df

In [ ]:
# # Mostrar cantidad de outliers por columna
# resumen = resumen_outliers(df)
# resumen

In [ ]:
# # ------------------------------------------------
# # Detectar columnas con valores continuos en `df`
# # ------------------------------------------------

# from pandas.api.types import is_float_dtype, is_integer_dtype

# # Umbral mínimo de valores únicos para considerar un entero como “continuo”
# INT_UNIQUE_THRESHOLD = 20

# continuous_features = []
# for col in df.columns:
#     series = df[col]
#     # Si es float, lo consideramos continuo
#     if is_float_dtype(series):
#         continuous_features.append(col)
#     # Si es entero y tiene muchos valores únicos, también lo consideramos continuo
#     elif is_integer_dtype(series) and series.nunique() > INT_UNIQUE_THRESHOLD:
#         continuous_features.append(col)

# print("Features continuas detectadas:")
# for feat in continuous_features:
#     print(f" - {feat} (dtype={df[feat].dtype}, únicos={df[feat].nunique()})")

In [ ]:
# # Define tus cortes manuales
# bins = [df["Quarterly_Tax"].min() - 1, 100, 150, 200, df["Quarterly_Tax"].max()]
# labels = [1, 2, 3, 4]

# # Crea la categoría
# df["Tax_RangeCat"] = pd.cut(df["Quarterly_Tax"], bins=bins, labels=labels).astype(int)

# # Muestra la relación entre Quarterly_Tax y la nueva categoría
# print(
#     df[["Quarterly_Tax", "Tax_RangeCat"]]
#     .drop_duplicates()
#     .sort_values("Quarterly_Tax")
#     .reset_index(drop=True)
# )

In [ ]:
# df

In [ ]:
# # Primero ponemos las continuas en un nuevo DataFrame
# df_toyota_continuas = df[continuous_features].copy()

# # Ahora veremos BoxPlots e Histogramas con sus curva de densidad
# histogram_por_lotes(df_toyota_continuas, 6)

In [ ]:
# boxplots_por_lotes(df_toyota_continuas, 6)

---

### Price

* **Distribución**: el histograma muestra una distribución con pico principal entre \$8 000 y \$12 000, con una larga cola hacia la derecha que llega hasta \$30 000. La curva de densidad confirma ese sesgo positivo.
* **Boxplot**: la mediana está cerca de \$10 000; el IQR va aproximadamente de \$8 000 a \$12 000. Hay muchos valores atípicos por encima de \$15 000, que corresponden a modelos o equipamientos premium.
* **Interpretación**: la mayoría de los coches se cotizan en un rango estrecho, pero existen unos pocos vehículos de alto precio que inflan la cola, por lo que conviene una transformación (por ejemplo, log) o tratar outliers antes de modelar.

---

### Age_08_04

- **Distribución**: el histograma es prácticamente creciente desde valores bajos hasta el máximo (\~80), y la densidad señala que hay un acumulado mayor en edades altas. No es simétrica: hay más coches "viejos".
- **Boxplot**: la mediana está en torno a 60–65, el IQR entre \~50 y \~70, con algunos autos muy recientes (cerca de 0) como outliers en la izquierda.
- **Interpretación**: la flota tiende a concentrarse en edades entre 50 y 80 años (o unidades de medida), con pocos vehículos nuevos. Al modelar, podría ser útil agrupar edades muy bajas o muy altas o usar técnicas robustas a outliers.

---

### KM

- **Distribución**: el histograma con KDE presenta un solo pico alrededor de 50 000–75 000 km y luego una cola larga hacia la derecha hasta >200 000 km.
- **Boxplot**: la mediana se sitúa cerca de 75 000 km; el IQR va de \~50 000 a \~100 000 km. Varios outliers por encima de 150 000 km.
- **Interpretación**: la mayoría de los vehículos tienen kilometrajes moderados, pero existe un subgrupo con uso intensivo. Para regresión podría convenir una transformación (raíz o log) y evaluar si recortar o imputar outliers.

---

### Weight

- **Distribución**: el histograma muestra un pico muy marcado entre 1 000 y 1 100 kg, con una cola derecha que llega hasta 1 600 kg; la densidad refleja un sesgo ligero a la derecha.
- **Boxplot**: la mediana ronda 1 050 kg, el IQR entre \~1 015 y \~1 100 kg, con algunos valores muy pesados como outliers.
- **Interpretación**: el peso es bastante homogéneo (la mayoría alrededor de \~1 050 kg), pero hay versiones más pesadas que conviene revisar (p. ej. carrocerías especiales o variante 4×4). Para modelar, quizá baste winsorizar esos pocos valores extremos.

---

**Resumen general**:

- **Precio** y **KM** presentan sesgo positivo y varios outliers altos.
- **Edad** está sesgada hacia valores altos, con pocos coches muy nuevos.
- **Peso** es la más concentrada, aunque con alguna cola derecha.


In [ ]:
# # Variables enteras

# df_toyota_enteras = df.copy()
# df_toyota_enteras.drop(
#     columns=[
#         "Price",
#         "Quarterly_Tax",
#         "Weight",
#         "KM",
#         "Mfr_Guarantee",
#         "BOVAG_Guarantee",
#         "ABS",
#         "Airbag_1",
#         "Airbag_2",
#         "Airco",
#         "Automatic_airco",
#         "Boardcomputer",
#         "CD_Player",
#         "Central_Lock",
#         "Met_Color",
#         "Powered_Windows",
#         "Power_Steering",
#         "Radio",
#         "Mistlamps",
#         "Sport_Model",
#         "Backseat_Divider",
#         "Metallic_Rim",
#         "Radio_cassette",
#         "Tow_Bar",
#         "Age_08_04_calculada",
#         "Fuel_Type",
#         "Automatic",
#     ],
#     axis=1,
#     inplace=True,
#     errors="ignore",  # Ignore if any column is missing
# )

In [ ]:
# df_toyota_enteras.describe().T

In [ ]:
# bar_por_lotes(df_toyota_enteras, 3)

# Eliminación de outliers en Price


In [ ]:
# # Eliminacion de outliers y/o Transformacion para varibles continuas

# histogram(df_toyota_continuas["Price"])
# boxplot(df_toyota_continuas["Price"])

Dicha distribucion presenta una sesgo hacia la izquierda por lo tanto hay que eliminar esos outliers, como el histograma lo presenta vemos que es una distrubucion que se acerca mucho a una `distribucion normal` lo que haremos es eliminar los outliers con el metodo llamado `z-core`


In [ ]:
# price = df_toyota_continuas["Price"].copy()
# _, mask_price = limpiar_outliers_z_core(price)
# price_limpio = price[mask_price]
# histogram(price_limpio)
# boxplot(price_limpio)

# print(len(price) - len(price_limpio))

# price_limpio_l, mask_price_l = limpiar_outliers_z_core(price_limpio)

# histogram(price_limpio_l)

# boxplot(price_limpio_l)

# print(len(price_limpio) - len(price_limpio_l))

# Eliminación de outliers en KM


In [ ]:
# histogram(df_toyota_continuas["KM"])
# boxplot(df_toyota_continuas["KM"])

dicha distribucion presenta una sesgo hacia la derecha por lo tanto hay que eliminar esos outliers, como el histograma lo presenta vemos que es una distribucion que se acerca mucho a una `distribucion normal` lo que haremos es eliminar los outliers con el metodo llamado `z-core`


In [ ]:
# km = df_toyota_continuas["KM"].copy()
# _, mask_km = limpiar_outliers_z_core(km)
# km_limpio = km[mask_km]
# histogram(km_limpio)
# boxplot(km_limpio)
# print(len(km) - len(km_limpio))

# Eliminacion de outliers en Weight


In [ ]:
# histogram(df_toyota_continuas["Price"])
# boxplot(df_toyota_continuas["Price"])

In [ ]:
# # 1. Copiar la serie de peso
# peso = df_toyota_continuas["Weight"].copy()

# # 2. Obtener la máscara de valores válidos con tu función de Z-score
# _, mask_peso = limpiar_outliers_z_core(peso)

# # 3. Filtrar los datos limpios
# peso_limpio = peso[mask_peso]

# # 4. Visualizar distribución y outliers
# histogram(peso_limpio)
# boxplot(peso_limpio)

# # 5. Imprimir cuántos registros se eliminaron
# print(f"Registros removidos: {len(peso) - len(peso_limpio)}")

# Eliminacion de outliers en Age


In [ ]:
# histogram(df_toyota_continuas["Age_08_04"])
# boxplot(df_toyota_continuas["Age_08_04"])

# Analisis Bivariado con dataframe original


In [ ]:
# from sklearn.preprocessing import LabelEncoder

# df["fuel_type_encoded"] = LabelEncoder().fit_transform(df["Fuel_Type"])

In [ ]:
# matriza = mostrar_matriz_correlacion(df)

Price vs. Mfg_Year (r ≈ +0.89)

Los autos más nuevos (año de fabricación alto) tienden a tener precios más elevados. Cada año adicional aumenta fuertemente el valor.

Price vs. Age_08_04 (r ≈ –0.88)

Edad y precio son espejo: a más antigüedad (edad alta) el precio baja. Cada unidad de edad adicional desploma el valor de manera proporcional.

Mfg_Year vs. Age_08_04 (r ≈ –0.98)

Lógicamente inversas: un coche más nuevo (año alto) tiene poca “edad” registrada.

Quarterly_Tax vs. Tax_RangeCat (r ≈ +0.93)

El impuesto trimestral está prácticamente definido por la categoría de rango fiscal; son casi sinónimos cuantitativos.

Airco vs. Automatic_airco (r ≈ +0.72)

Tener aire acondicionado se superpone en gran medida con la versión “automática” de ese aire, indica redundancia de ambas variables.

Boardcomputer vs. Mfg_Year (r ≈ +0.72)

Los coches más modernos casi siempre traen ordenador de a bordo, reflejando que ese equipamiento se incorporó en modelos recientes.

Central_Lock vs. Powered_Windows (r ≈ +0.88)

El cierre centralizado y las ventanillas eléctricas suelen venir juntos en el mismo nivel de acabado.

Radio vs. Radio_cassette (r ≈ +0.99)

HP vs. Mfg_Year (r ≈ +0.72)
Modelos más nuevos tienden a tener más potencia; esta alta colinealidad puede inflar la varianza de los coeficientes si ambas variables entran al mismo tiempo.

Casi todos los coches con radio incorporan también reproductor de casete; las dos variables miden esencialmente el mismo equipamiento.


In [ ]:
# # Realizamos feature selection de acuerdo a la alta correlacion
# cols_to_drop = [
#     "Mfg_Year",
#     "Age_08_04",
#     "Tax_RangeCat",
#     "Automatic_airco",
#     "Boardcomputer",
#     "Powered_Windows",
#     "Radio_cassette",
#     "Fuel_Type",
#     "Quarterly_Tax",
#     "BOVAG_Guarantee",
#     "Duplicado_de",
# ]

# df_clean = df.drop(columns=cols_to_drop)
# df_clean

In [ ]:
# matriz = mostrar_matriz_correlacion(df_clean)

In [ ]:
# df.columnas

In [ ]:
# df.dtypes

In [ ]:
# from pathlib import Path


# clean_path = Path("data/clean_df.csv").resolve()
# df_clean.to_csv(clean_path, index=False)
# clean_path